In [171]:
import pandas as pd
import os
os.chdir("C:/Users/pauli/Documents/M2/Text Mining/projet/Disney-Text-Mining/fichiers")
mypath = os.getcwd()

# table date 


In [172]:
df = pd.read_csv(str(mypath)+"/df_clean/df_clean.csv",sep=';')


In [173]:
df["id_date"] = df["date"].astype(str)
df["id_date"] = df["id_date"].str.replace("-","")

In [174]:
champs_date = ["month_str","month_num","year","date", "id_date"]

In [175]:
df_date = df[champs_date].copy()
df_date.drop_duplicates(keep='first', inplace=True)
df_date.reset_index(drop=True, inplace=True)
df_date.to_csv(str(mypath)+"/tables/date.csv",sep=';', index=False, encoding="utf-8-sig")

# Client

In [176]:
champs_client = ["Country", "nuitee", "traveler_infos", "review_title", "positive_review", "negative_review", "usefulness_review"]

In [177]:
df_client = df[champs_client].copy()
df_client.rename(columns={'Country': 'country', 'nuitee': 'nb_night'}, inplace=True)
import uuid
df_client['id_client'] = df_client.apply(lambda _: uuid.uuid4(), axis=1)
df_client.to_csv(str(mypath)+"/tables/client.csv",sep=';', index=False, encoding="utf-8-sig")

# hotel

In [178]:
champs_hotel = ["hotel"]

In [179]:
df_hotel = df[champs_hotel].copy()
df_hotel.drop_duplicates(keep='first', inplace=True)
df_hotel.dropna(inplace=True)
import uuid
df_hotel['id_hotel'] = df_hotel.apply(lambda _: uuid.uuid4(), axis=1)
df_hotel.to_csv(str(mypath)+"/tables/hotel.csv",sep=';', index=False, encoding="utf-8-sig")

# chambre

In [180]:
champs_chambre = ["room_type", "hotel"]

In [181]:
df_room = df[champs_chambre].copy()
df_room.drop_duplicates(keep='first', inplace=True)
import uuid
df_room['id_room'] = df_room.apply(lambda _: uuid.uuid4(), axis=1)
df_room.reset_index(drop=True, inplace=True)

#merge pour avoir id_hotel
df_room=df_room.merge(df_hotel, on='hotel')
df_room.drop(columns=['hotel'], inplace=True)
df_room.to_csv(str(mypath)+"/tables/room.csv",sep=';', index=False, encoding="utf-8-sig")

# Reservation

In [182]:
champs_res=["grade_review", "level_grade_review", "id_date", "room_type"]

In [183]:
df_res = df[champs_res].copy()
import uuid
df_res.reset_index(drop=True, inplace=True)

#merge pour avoir tous les id
#doit avoir 11219 lignes
df_res['id_reservation'] = df_res.apply(lambda _: uuid.uuid4(), axis=1)
df_res = pd.merge(df_res, df_room[["id_room", "room_type"]], how="left", on = ["room_type"])
df_res.drop_duplicates(subset=['id_reservation'], inplace=True)
df_res.reset_index(drop=True, inplace=True)

df_res.drop(columns=['room_type'], inplace=True)
df_res = pd.concat([df_res, df_client['id_client']], axis=1) 
df_res.to_csv(str(mypath)+"/tables/reservation.csv",sep=';', index=False, encoding="utf-8-sig")